In [13]:
%load_ext autoreload
%autoreload 2
%cd C:\MAD4AG
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\MAD4AG


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import warnings

warnings.filterwarnings('ignore')

In [15]:
file_name = f'./dbs/intermediate/stops_2.parquet'

#Read each batchs file
df = pd.read_parquet(file_name)

In [16]:
print('The number of stops: ',len(df))
df = df[df.holiday_s != 1]
df = df[df.weekday_s == 1]
df = df.drop(['holiday_s', 'weekday_s'], axis=1)

print('The number of stops in weekday and non-holiday: ', len(df))

The number of stops:  29006512
The number of stops in weekday and non-holiday:  13453097


## drop stop points that are detected in the home cluster.

In [17]:
df_h = pd.read_parquet(f'./dbs/intermediate/home_inference.parquet')
df_h.drop_duplicates(subset='uid', keep='first', inplace=True)

In [18]:
aaa = df.head(1000)

In [23]:
df_1 = df.loc[:,['uid', 'localtime', 'loc', 'h_s', 'dur', 'seq', 'cluster', 'Deso', 'ANDAMAL_1', 'distance']]


df_1 = pd.merge(df_1, df_h[['uid','cluster','score']], on= ['uid','cluster'], how="left")
df_1 = df_1[df_1.score.isnull()]
df_1.drop(columns=['score'], inplace=True)

### repeat each row (stop) based on stay duration.
### e.g., if stay in a stop longer than 30 min, make two stops

In [24]:
df_stop = df_1.copy()

In [25]:
df_stop['dur_rp'] = (df_stop['dur'] / 30) + 1
df_stop['dur_rp'] = df_stop['dur_rp'].astype(int)
df_stop = pd.DataFrame(np.repeat(df_stop.values, df_stop['dur_rp'].tolist(), axis=0), columns=df_stop.columns)

df_stop.drop(['dur_rp'], inplace=True, axis=1)

### grouping and auto increment in index based stay


In [26]:
df_stop['stay_seq'] = df_stop.groupby(['uid', 'seq']).cumcount()
df_stop['timestamp'] = pd.to_datetime(df_stop['localtime']) + (pd.Timedelta(minutes=30) * df_stop['stay_seq'])
df_stop['hour'] = df_stop['timestamp'].dt.hour

In [28]:
aaa = df_stop.head(1000)